In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 4.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [4]:
ls

 ColabDriveGitHub.ipynb     day4.ipynb        day5.ipynb   dw_matrix/
'Copy of Untitled0.ipynb'   day4_meta.ipynb   deep.ipynb   Untitled0.ipynb


In [5]:
ls data

ls: cannot access 'data': No such file or directory


In [6]:
 cd dw_matrix

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/shoes_prices.csv',low_memory=False)  

In [9]:
df.shape

(19387, 48)

In [10]:
df['prices_amountmin'] = pd.to_numeric(df['prices_amountmin'], errors='coerce')
df_usd = df[df.prices_currency=="USD"].copy()
df_usd.shape

(18524, 48)

In [0]:
def run_model (feats, model = DecisionTreeRegressor(max_depth=5)):
  X=df_usd_filter[feats].values
  y = df_usd_filter['prices_amountmin'].values
  scores = cross_val_score(model,X,y,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
filter_max = np.percentile(df_usd['prices_amountmin'],99)
filter_max
df_usd_filter = df_usd[df_usd['prices_amountmin']<filter_max]

In [14]:
df_usd_filter['brand_cat'] = df_usd_filter['brand'].factorize()[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
run_model(['brand_cat'] )

(-58.38655694633361, 4.223555478221712)

In [21]:
model = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
run_model(['brand_cat'], model )

(-57.31783843165656, 4.181246596160967)

In [20]:
df_usd_filter['brand_cat'] = df_usd_filter['brand'].map(lambda x:str(x).lower()).factorize()[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
from ast import literal_eval
from tqdm import tqdm_notebook

In [25]:
literal_eval('[{"key":"Gender","value":["Men"]}]')[0]['value']

['Men']

In [31]:
def parse_features(x):
  ret = {}
  if str(x) == 'nan': return ret
  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    ret[key] = value
  return ret

df_usd_filter['features_parsed'] = df_usd_filter['features'].map(parse_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [33]:
df_usd_filter['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [34]:
keys = set()
df_usd_filter['features_parsed'].map(lambda x:keys.update(x.keys()))
len(keys)

476

In [0]:
def get_name_feat(key):
  return 'feat_'+key

In [36]:
for key in tqdm_notebook(keys):
   df_usd_filter[get_name_feat(key)] = df_usd_filter.features_parsed.map(lambda feats : feats[key] if key in feats else np.nan)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
df_usd_filter.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_����������������', 'feat_safety features',
       'feat_country of manufacturer', 'feat_net weight', 'feat_wheel type',
       'feat_has adaptive lenses', 'feat_hood', 'feat_clothing size',
       'feat_hooded', 'feat_features'],
      dtype='object', length=526)

In [38]:
df_usd_filter[df_usd_filter['feat_athlete'].isnull()].shape

(18272, 526)

In [39]:
df_usd_filter.shape

(18280, 526)

In [0]:
keys_stat = {}
for key in keys :
  keys_stat[key] = df_usd_filter[False == df_usd_filter[get_name_feat(key)].isnull()].shape[0]/df_usd_filter.shape[0]*100

In [43]:
keys_stat

{'100% adidas authentic': 0.005470459518599562,
 'accessory type': 0.1422319474835886,
 'adidas': 0.005470459518599562,
 'adjustable': 0.34463894967177244,
 'age': 0.6400437636761488,
 'age end': 0.4431072210065646,
 'age gender group': 0.12582056892778995,
 'age group': 27.64770240700219,
 'age range': 0.5470459518599562,
 'age segment': 0.1422319474835886,
 'age start': 0.4431072210065646,
 'airport friendly': 0.005470459518599562,
 'alarm': 0.23522975929978115,
 'amazonbestsellersrank': 0.06017505470459519,
 'animal type': 0.005470459518599562,
 'antiscratch lens coating': 0.005470459518599562,
 'applicable': 0.02188183807439825,
 'arm': 0.6236323851203501,
 'article': 0.005470459518599562,
 'assembled in country of origin': 2.199124726477024,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'assembled product weight': 1.5536105032822756,
 'athlete': 0.0437636761487965,
 'atpv arc rating': 0.005470459518599562,
 'attachment': 0.03282275711159737,
 'audience': 0.2352

In [48]:
{k:v for k,v in keys_stat.items() if v > 30 }

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [49]:
df_usd_filter['feat_brand_cat'] = df_usd_filter['feat_brand'].factorize()[0]
df_usd_filter['feat_color_cat'] = df_usd_filter['feat_color'].factorize()[0]
df_usd_filter['feat_gender_cat'] = df_usd_filter['feat_gender'].factorize()[0]
df_usd_filter['feat_manufacturer part number_cat'] = df_usd_filter['feat_manufacturer part number'].factorize()[0]
df_usd_filter['feat_material_cat'] = df_usd_filter['feat_material'].factorize()[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [50]:
df_usd_filter[df_usd_filter.brand!=df_usd_filter.feat_brand].shape

(18228, 531)

In [52]:
feats = ['brand_cat','feat_brand_cat','feat_color_cat','feat_gender_cat','feat_manufacturer part number_cat','feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats,model)

(-57.10982896712716, 4.260154124512923)

In [53]:
feats = ['feat_brand_cat','feat_color_cat','feat_gender_cat','feat_manufacturer part number_cat','feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats,model)

(-60.452852027586346, 4.108819231443576)

In [55]:
X = df_usd_filter[feats].values
y = df_usd_filter['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)


Weight,Feature
0.1037 ± 0.0106,feat_material_cat
0.0834 ± 0.0013,feat_gender_cat
0.0337 ± 0.0036,feat_color_cat
0.0312 ± 0.0031,feat_brand_cat
0.0024 ± 0.0006,feat_manufacturer part number_cat


In [57]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats,model)

(-57.13442449775489, 4.210643607005348)

In [60]:
df_usd_filter[df_usd_filter['brand'] == 'nike'].features_parsed.head().values

array([{'style': 'running, cross training', 'material': 'synthetic', 'condition': 'new with box'},
       {'sport': 'baseball & softball', 'type': 'cleats', 'condition': 'new with box'},
       {'condition': 'new with box', 'type': 'cleats'},
       {'style': 'running, cross training, walking', 'condition': 'new without box'},
       {'sport': 'soccer', 'type': 'cleats'}], dtype=object)

In [61]:
df_usd_filter['feat_sport_cat'] = df_usd_filter['feat_sport'].factorize()[0]
df_usd_filter['feat_style_cat'] = df_usd_filter['feat_style'].factorize()[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [63]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat','feat_sport_cat','feat_style_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats,model)

(-57.04713722381693, 4.2092315118692785)

In [64]:
X = df_usd_filter[feats].values
y = df_usd_filter['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

Weight,Feature
0.2608 ± 0.0111,brand_cat
0.1046 ± 0.0083,feat_material_cat
0.0455 ± 0.0046,feat_gender_cat
0.0216 ± 0.0012,feat_brand_cat
0.0070 ± 0.0013,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat
